# Load Data

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from result_saver import SaverProvider
provider = SaverProvider()

In [4]:
import numpy as np
from Scratch import metadata_loader

DEVICE = 'ibm_sherbrooke'

md = metadata_loader(_extract=True, _drop_inutile=True)
md = md[md["job_status"] == "JobStatus.DONE"]
md = md[md["notebook_name"] == "bigger_rep_codes"]
max_distance = int(max(md.distance))
max_distance = 30
md = md[md["distance"] == max_distance]
md = md[md["backend_name"]==DEVICE]

md = md[:2]

md

,creation_date,notebook_name,backend_name,job_id,tags,shots,tags_xp,sampled_state,num_qubits,job_status,extra,optimization_level,code,distance,rounds,logical,layout,descr
258,2023-10-30 09:45:17.340386+01:00,bigger_rep_codes,ibm_sherbrooke,cmzpt78vcq70008qf1ag,[],1111.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,30.0,30,0,_is_hex=True,Run bigger Repetition codes v4: new distances ...
76,2023-10-29 14:47:58.814875+01:00,bigger_rep_codes,ibm_sherbrooke,cmz653m3r3vg008wf9j0,[],1111.0,NaN,NaN,NaN,JobStatus.DONE,NaN,NaN,RepetitionCodeCircuit,30.0,30,1,_is_hex=True,Run bigger Repetition codes


In [5]:
memories = {}
for job_id, logical in zip(md.job_id, md.logical):
    mmr_name = f"mmr_log_{logical}"
    memories[mmr_name] = provider.retrieve_job(job_id).result().get_memory()

memory = memories["mmr_log_1"][:]

# UF Decoder

In [6]:
import pickle
from qiskit_qec.circuits import RepetitionCodeCircuit
from soft_info import get_repcode_layout, get_KDEs, get_counts

layout = get_repcode_layout(distance=max_distance, backend=provider.get_backend("ibm_nazca"), _is_hex=True)
kde_dict, scaler_dict = get_KDEs(provider, 'cmz653m3r3vg008wf9j0')
code = RepetitionCodeCircuit(max_distance, max_distance)


# PyMatching

In [7]:
import stim
import pymatching

d = max_distance
T = max_distance

circuit = stim.Circuit.generated("repetition_code:memory",
                                 distance=d,
                                 rounds=T,
                                 after_clifford_depolarization=0.1)

model = circuit.detector_error_model(decompose_errors=True)
matching = pymatching.Matching.from_detector_error_model(model)


In [9]:
import cProfile
import pstats


from tqdm import tqdm
from soft_info import get_counts, soft_reweight_pymatching, counts_to_det_syndr, draw_matching_graph, reweight_edges_to_one



def main():
    VERBOSE = False

    #actual_observables = np.array([[False]]) # hardcoded, can be retrieved
    num_errors = 0

    i = 0
    w_idx_lst = []
    for shot in tqdm(range(len(memory))[:20]):
        i += 1
        IQ_data = memory[shot]

        counts = get_counts([IQ_data], kde_dict, scaler_dict, layout, T, verbose=False)
        count_key = next(iter(counts.keys()))
        
        actual_observables = [(int(count_key[0])+1)%2]
        soft_reweight_pymatching(matching, d, T, IQ_data, kde_dict, layout, scaler_dict, common_measure=0.01, verbose=False)  
        #reweight_edges_to_one(matching)

        array_processed_string = counts_to_det_syndr(count_key, _resets=False, verbose=False)

        predicted_observables = matching.decode(array_processed_string)

        if predicted_observables == actual_observables: #== [0]:
            continue
        num_errors += 1
        #print("actual_observables:", actual_observables)
        #print("predicted_observables:", predicted_observables)
        print("num_errors:", num_errors)

        #print(f"Wrong decoding at index {i}")
        w_idx_lst.append(i)

        
        if VERBOSE:
            print("Count key:", count_key)
            print("count_string_to_syndromes:", array_processed_string)
            print("actual_observables:", actual_observables)
            print("predicted_observables:", predicted_observables)

        
        if VERBOSE:
            matched_edges = matching.decode_to_edges_array(array_processed_string)
            print("matched_edges: ", matched_edges)
            print("Estimated flip:", predicted_observables)
            
        if VERBOSE:
            draw_matching_graph(matching, d, T, syndromes=array_processed_string, matched_edges=matched_edges, figsize=(10, 10))

    print("Num errors:", num_errors)





In [10]:
if __name__ == "__main__":
    profiler = cProfile.Profile()
    profiler.enable()
    main()
    profiler.disable()
    stats = pstats.Stats(profiler).sort_stats('cumtime')
    stats.print_stats()


100%|██████████| 20/20 [00:16<00:00,  1.24it/s]

Num errors: 0
         15342177 function calls (15162160 primitive calls) in 16.196 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001   16.196   16.196 /var/folders/_3/n7zzdff5239886c9gmhrk6yr0000gn/T/ipykernel_62572/3074820388.py:10(main)
       20    0.040    0.002   15.965    0.798 /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/src/soft_info/Probabilities/probabilities.py:46(get_counts)
    18000    0.072    0.000   15.915    0.001 /Users/mha/My Drive/Desktop/Studium/Physik/MSc/Semester 3/IBM/IBM GIT/Soft-Info/src/soft_info/Probabilities/probabilities.py:15(estimate_outcome)
    36000    0.177    0.000   14.473    0.000 /Users/mha/.local/share/virtualenvs/Soft-Info-Sk8aHGSa/lib/python3.11/site-packages/sklearn/neighbors/_kde.py:245(score_samples)
    36000   10.141    0.000   12.002    0.000 {method 'kernel_density' of 'sklearn.neighbors._kd_tree.BinaryTree' ob